In [26]:
from sklearn.model_selection import RepeatedStratifiedKFold,GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,VarianceThreshold
import numpy as np
import pandas as pd
from pprint import pprint
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import numpy as np
from sklearn import linear_model, svm
import re
from sklearn.metrics import roc_curve, auc,f1_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
#s = "../braindata/data_1_mor_select_100.csv"
import os          
from sklearn import linear_model, svm
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC


In [51]:
import pandas as pd
import numpy as np
import logging
import time
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV



os.getcwd()
os.chdir('/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/idp_jiook/data/adni')
dd =pd.read_csv("combine_new.csv",header=0)
print(dd.shape)
import csv


with open('combine_new.csv', 'r') as f:
    d_reader = csv.DictReader(f)

    #get fieldnames from DictReader object and store in list
    headers = d_reader.fieldnames
data=np.array(dd)
#print(data.shape)
idx_IN_columns = np.append(np.array([9,17]),np.array(range(18,data.shape[1])))
print(idx_IN_columns)
X=data[:,idx_IN_columns]
#features=data[:,11:data.shape[1]]
#features = features.transpose()
X = stats.zscore(X)
#print(features.shape)
y=data[:,6]
#/ 6:AD-normal / 7:AD-MCI / 8:MCI-normal 




ind_num=np.isnan(y)
# print(ind_num.shape)


y_no_nan = y[~ind_num]

X_no_nan = X[~ind_num,:]

       # print(y.shape)

y=y_no_nan
X=X_no_nan
feature_num_all=[]
lr_all_feature=[]
svm_all_feature=[]
lr_fls_feature=[]
svm_fls_feature=[]
base_labels= []

np.isnan(X).any()

X[np.isnan(X)] = np.median(X[~np.isnan(X)])

selector = VarianceThreshold()
selector.fit_transform(X)
print(X.shape)

model = RandomForestClassifier()
parameters = [{"n_estimators": [50,100,250, 500, 1000,5000]}]
 
# Returns the best configuration for a model using crosvalidation
# and grid search
def best_config(model, parameters, train_instances, judgements):
    clf = GridSearchCV(model, parameters, cv=10,
                       scoring="accuracy", verbose=5, n_jobs=4)
    clf.fit(train_instances, judgements)
    best_estimator = clf.best_estimator_
    #log_info('Best hyperparameters: ' + str(clf.best_params_))
    return [str(clf.best_params_), clf.best_score_, best_estimator]


best_config(model,parameters,X,y)



(179, 34728)
[    9    17    18 ..., 34725 34726 34727]


/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:2247: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


(119, 34712)
Fitting 10 folds for each of 6 candidates, totalling 60 fits
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.923077 -   0.5s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.833333 -   0.5s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.750000 -   0.5s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.666667 -   0.4s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.833333 -   0.4s
[CV

[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    1.5s


[CV] ....................... n_estimators=100, score=0.923077 -   0.7s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.916667 -   0.7s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.750000 -   0.7s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.583333 -   0.7s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.833333 -   0.6s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.916667 -   0.6s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.833333 -   0.6s
[CV] ....................... n_estimators=100, score=0.750000 -   0.6s
[CV] n

[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:  1.5min finished


["{'n_estimators': 1000}",
 0.8487394957983193,
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False)]

In [52]:
# Returns the best model from a set of model families given
# training data using cross-validation.
def best_model(classifier_families, train_instances, judgements):
    best_quality = 0.6
    best_classifier = None
    classifiers = []
    for name, model, parameters in classifier_families:
        classifiers.append(best_config(model, parameters,
                                       train_instances,
                                       judgements))
 
    for name, quality, classifier in classifiers:
        if (quality > best_quality):
            
            best_quality = quality
            best_classifier = [name, classifier]
 
    return best_classifier


# List of candidate family classifiers with parameters for grid
# search [name, classifier object, parameters].
def candidate_families():
    candidates = []
    svm_tuned_parameters = [{'kernel': ['poly'],
                             'degree': [1, 2, 3, 4]}]
    candidates.append(["SVM", SVC(C=10), svm_tuned_parameters])
    
    rf_tuned_parameters = [{"n_estimators": [50, 100,250, 500, 1000]}]
    candidates.append(["RandomForest",
                       RandomForestClassifier(n_jobs=-1),
                       rf_tuned_parameters])
 
    knn_tuned_parameters = [{"n_neighbors": [1, 3, 5, 10, 20]}]
    candidates.append(["kNN", KNeighborsClassifier(),
                       knn_tuned_parameters])
 
    return candidates


candidates=candidate_families()
best_model(candidates,X,y)


Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] degree=1, kernel=poly ...........................................
[CV] degree=1, kernel=poly ...........................................
[CV] degree=1, kernel=poly ...........................................
[CV] degree=1, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.923077 -   0.8s
[CV] degree=1, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.833333 -   0.8s
[CV] degree=1, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.666667 -   0.8s
[CV] degree=1, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.666667 -   0.8s
[CV] degree=1, kernel=poly ...........................................
[CV] .................. degree=1, kernel=poly, score=0.750000 -   0.8s
[CV] degree=1, k

[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.6s


[CV] .................. degree=2, kernel=poly, score=0.666667 -   0.8s
[CV] degree=2, kernel=poly ...........................................
[CV] .................. degree=2, kernel=poly, score=0.583333 -   0.8s
[CV] degree=2, kernel=poly ...........................................
[CV] .................. degree=2, kernel=poly, score=0.666667 -   0.8s
[CV] degree=2, kernel=poly ...........................................
[CV] .................. degree=2, kernel=poly, score=0.583333 -   0.8s
[CV] degree=2, kernel=poly ...........................................
[CV] .................. degree=2, kernel=poly, score=0.666667 -   0.7s
[CV] degree=3, kernel=poly ...........................................
[CV] .................. degree=2, kernel=poly, score=0.583333 -   0.7s
[CV] degree=3, kernel=poly ...........................................
[CV] .................. degree=2, kernel=poly, score=0.636364 -   0.8s
[CV] degree=3, kernel=poly ...........................................
[CV] .

[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    8.3s finished


Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.923077 -   0.4s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.833333 -   0.4s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.750000 -   0.5s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.583333 -   0.5s
[CV] n_estimators=50 .................................................
[CV] ........................ n_estimators=50, score=0.750000 -   0.4s
[CV] n_estimator

...

Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV] n_neighbors=1 ...................................................
[CV] n_neighbors=1 ...................................................
[CV] n_neighbors=1 ...................................................
[CV] n_neighbors=1 ...................................................
[CV] .......................... n_neighbors=1, score=0.846154 -   0.2s
[CV] n_neighbors=1 ...................................................
[CV] .......................... n_neighbors=1, score=0.750000 -   0.2s
[CV] n_neighbors=1 ...................................................
[CV] .......................... n_neighbors=1, score=0.583333 -   0.2s
[CV] n_neighbors=1 ...................................................
[CV] .......................... n_neighbors=1, score=0.666667 -   0.3s
[CV] n_neighbors=1 ...................................................
[CV] .......................... n_neighbors=1, score=0.583333 -   0.2s
[CV] n_neighbors

[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.9s


[CV] n_neighbors=3 ...................................................
[CV] .......................... n_neighbors=3, score=0.666667 -   0.2s
[CV] n_neighbors=3 ...................................................
[CV] .......................... n_neighbors=3, score=0.666667 -   0.2s
[CV] n_neighbors=3 ...................................................
[CV] .......................... n_neighbors=3, score=0.583333 -   0.2s
[CV] .......................... n_neighbors=3, score=0.583333 -   0.2s
[CV] n_neighbors=3 ...................................................
[CV] n_neighbors=5 ...................................................
[CV] .......................... n_neighbors=3, score=0.583333 -   0.2s
[CV] n_neighbors=5 ...................................................
[CV] .......................... n_neighbors=3, score=0.909091 -   0.2s
[CV] n_neighbors=5 ...................................................
[CV] .......................... n_neighbors=3, score=0.727273 -   0.2s
[CV] n

[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    3.5s finished


["{'n_estimators': 100}",
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False)]

In [49]:
# clf = ExtraTreesClassifier(n_estimators=250,
#                               random_state=0)
# clf = clf.fit(X, y)
# importances = clf.feature_importances_
# importances
# indices = np.argsort(importances)[::-1]
# #print("Feature ranking:")

# #for f in range(X.shape[1]):
# #    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
# n_features = [10,30,50,70,80,100,1000,2000,13000,18000,10000,20000,3000,30000,4000,500,5000,6000,7000,8000,9000,15000,25000]
# #n_features = [1000]
# n_features.sort()

# for i in n_features:
    
       
#     index=indices[0:i]
#     print(index.shape)
#     features=X[:,index]

#     feature_num=features.shape[1]
#     print(feature_num)
#     lr_all_scores = []
#     lr_f1s = []
#     svm_all_accs = []
#     svm_all_scores = []
#     svm_f1s = []
#     #base_labels = []
#     #for runs in range(1):
#         lr_accuracies = []
#         lr_scores = []
#         svm_accuracies = []
#         svm_scores = []
#         strat_labels = []
        
logistic = linear_model.LogisticRegression(C=1e5)
rbf = svm.SVC(C=10,kernel='linear',gamma=0.01)

skf=RepeatedStratifiedKFold(n_splits=10, n_repeats=10)
lr_all_accs = []
lr_all_scores = []
lr_f1s = []
svm_all_accs = []
svm_all_scores = []
svm_f1s = []

lr_accuracies = []
lr_scores = []
svm_accuracies = []
svm_scores = []
strat_labels = []

        
for train_index, test_index in skf.split(X, y):

        train_data, test_data = X[train_index], X[test_index]
        train_labels, test_labels = y[train_index], y[test_index]
        strat_labels = np.append(strat_labels, test_labels)

        clf = ExtraTreesClassifier(n_estimators=250, random_state=0)
        clf = clf.fit(train_data, train_labels)
        importances = clf.feature_importances_
        importances
        indices = np.argsort(importances)[::-1]
        #print("Feature ranking:")
        print(train_labels.shape)
        #for f in range(X.shape[1]):
        #    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
        n_features = [10,30,50,70,80,100,1000,2000,13000,18000,10000,20000,3000,30000,4000,500,5000,6000,7000,8000,9000,15000,25000]
        #n_features = [1000]
        n_features.sort()



        for i in n_features:


            index=indices[0:i]
            print(index.shape)
            features=train_data[:,index]

            feature_num=features.shape[1]
            #print(feature_num)


            #train_data, test_data = features[train_index], features[test_index]
            #train_labels, test_labels = y[train_index], y[test_index]


            log = logistic.fit(train_data[:,index], train_labels)
            print(train_data[:,index].shape)
            log_prob = log.decision_function(test_data[:,index])
            log_acc = log.score(test_data[:,index], test_labels)

            #f1 calculation
            y_pred = logistic.predict(test_data[:,index])
            log_f1 = f1_score(test_labels, y_pred)

            lr_accuracies = np.append(lr_accuracies, log_acc)
            lr_scores = np.append(lr_scores, log_prob)
            lr_f1s = np.append(lr_f1s, log_f1)
            print('Logistic Regression Accuracy: %f' % log_acc)

            rbf = rbf.fit(train_data[:,index], train_labels)
            svm_acc = rbf.score(test_data[:,index], test_labels)
            svm_prob = rbf.decision_function(test_data[:,index])

            #f1 calculation
            y_pred = rbf.predict(test_data[:,index])
            svm_f1 = f1_score(test_labels, y_pred)
            print('SVM Accuracy: %f' % svm_acc)
            svm_accuracies = np.append(svm_accuracies, svm_acc)
            svm_scores = np.append(svm_scores, svm_prob)
            svm_f1s = np.append(svm_f1s, svm_f1)
            #print('SVM f1: %f' % svm_f1)


        lr_all_scores = np.append(lr_all_scores, lr_scores)
        lr_fold_avg = np.mean(lr_accuracies)
        lr_all_accs = np.append(lr_all_accs, lr_fold_avg)
        svm_all_scores = np.append(svm_all_scores, svm_scores)
        svm_fold_avg = np.mean(svm_accuracies)
        svm_all_accs = np.append(svm_all_accs, svm_fold_avg)
            #print('Logistic Regression Accuracy: %f' % log_acc_avg)
                    #print('SVM Regression Accuracy: %f' % svm_acc_avg)
        feature_num_all=np.append(feature_num_all,feature_num)
        #print(feature_num_all)
# print(np.mean(lr_all_accs))
        print(np.mean(svm_all_accs))      
        lr_all_feature=np.append(lr_all_feature,np.mean(lr_all_accs))
        svm_all_feature=np.append(svm_all_feature,np.mean(svm_all_accs))
        lr_fls_feature=np.append(lr_fls_feature,np.mean(lr_f1s))
        svm_fls_feature=np.append(svm_fls_feature,np.mean(svm_f1s))
            #base_labels_all = np.append(base_labels_all,strat)
        base_labels = np.append(base_labels, np.mean(strat_labels))

(117,)
(10,)
(117, 10)
Logistic Regression Accuracy: 0.785714
SVM Accuracy: 0.714286
(30,)
(117, 30)
Logistic Regression Accuracy: 0.500000
SVM Accuracy: 0.500000
(50,)
(117, 50)
Logistic Regression Accuracy: 0.714286
SVM Accuracy: 0.714286
(70,)
(117, 70)
Logistic Regression Accuracy: 0.714286
SVM Accuracy: 0.714286
(80,)
(117, 80)
Logistic Regression Accuracy: 0.857143
SVM Accuracy: 0.857143
(100,)
(117, 100)
Logistic Regression Accuracy: 0.714286
SVM Accuracy: 0.714286
(500,)
(117, 500)
Logistic Regression Accuracy: 0.571429
SVM Accuracy: 0.571429
(1000,)
(117, 1000)
Logistic Regression Accuracy: 0.571429
SVM Accuracy: 0.642857
(2000,)
(117, 2000)
Logistic Regression Accuracy: 0.785714
SVM Accuracy: 0.714286
(3000,)
(117, 3000)
Logistic Regression Accuracy: 0.785714
SVM Accuracy: 0.714286
(4000,)
(117, 4000)
Logistic Regression Accuracy: 0.785714
SVM Accuracy: 0.714286
(5000,)
(117, 5000)
Logistic Regression Accuracy: 0.785714
SVM Accuracy: 0.714286
(6000,)
(117, 6000)
Logistic Regr

(118, 3000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(4000,)
(118, 4000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(5000,)
(118, 5000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(6000,)
(118, 6000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.692308
(7000,)
(118, 7000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.769231
(8000,)
(118, 8000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.692308
(9000,)
(118, 9000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(10000,)
(118, 10000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(13000,)
(118, 13000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(15000,)
(118, 15000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(18000,)
(118, 18000)
Logistic Regression Accuracy: 0.692308
SVM Accuracy: 0.692308
(20000,)
(118, 20000)
Logistic Regression Accuracy: 0.692308
SVM Accuracy: 0.692308
(25000,)
(118,

SVM Accuracy: 0.615385
(13000,)
(118, 13000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(15000,)
(118, 15000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(18000,)
(118, 18000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(20000,)
(118, 20000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(25000,)
(118, 25000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
(30000,)
(118, 30000)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.615385
0.647334357997
(118,)
(10,)
(118, 10)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.538462
(30,)
(118, 30)
Logistic Regression Accuracy: 0.692308
SVM Accuracy: 0.692308
(50,)
(118, 50)
Logistic Regression Accuracy: 0.615385
SVM Accuracy: 0.538462
(70,)
(118, 70)
Logistic Regression Accuracy: 0.538462
SVM Accuracy: 0.538462
(80,)
(118, 80)
Logistic Regression Accuracy: 0.538462
SVM Accuracy: 0.538462
(100,)
(118, 100)
Logistic Regression Accuracy: 0.538462
SVM 

In [53]:
print(svm_all_feature)

[]


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot( (feature_num_all), lr_all_feature*100, lw=5, label='Logistic Regression')
plt.plot( (feature_num_all), svm_all_feature*100, lw=5, label='SVM')
plt.xlim([50, 30000])
plt.ylim([0, 100])
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.title('MC converter vs non-converter')
plt.legend(loc="lower right")
plt.savefig('/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/idp_jiook/data/MCIconvert_combine.pdf')
plt.show()

In [14]:
print("svm accuracy: {}".format(np.mean(svm_acc)))
print("svm f1: {}".format(np.mean(svm_f1s)))
print(svm_f1)


svm accuracy: 0.9473684210526315
svm f1: 0.8695652173913044
0.96


In [1]:
#print(svm_f1s)

# print("LR accuracy Avg: {}".format(np.mean(lr_all_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
# print("LR accuracy Standard Deviation: {}".format(np.std(lr_all_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))

# print("LR f1s Avg : {}".format(np.mean( lr_fls_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
# print("LR f1s Standard Deviation: {}".format(np.std(lr_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))

# print("SVM Avg: {}".format(np.mean(svm_all_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
# print("SVM Standard Deviation: {}".format(np.std(svm_all_accs)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))

# print("SVM f1s Avg : {}".format(np.mean( svm_fls_feature )),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
# print("SVM f1s Standard Deviation: {}".format(np.std(svm_fls_feature)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
lrindex=np.argmax(lr_all_feature)
# print("LR Maximum feature number is: {}".format(feature_num_all[lrindex]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
svmindex=np.argmax(svm_all_feature)
# print("SVM Maximum feature is number : {}".format(feature_num_all[svmindex]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))


# clf = ExtraTreesClassifier(n_estimators=250,
#                               random_state=0)
# clf = clf.fit(X, y)
# importances = clf.feature_importances_
# #importances
# indices = np.argsort(importances)[::-1]
print("Feature ranking:")

for f in range(100):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f],headers[indices[f]],importances[indices[f]]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMI_rank.txt", "a"))

    
#################################################################################################################    
from sklearn import metrics

max_features = feature_num_all[svmindex]
print(max_features)
lr_all_accs = []
lr_all_scores = []
lr_f1s = []
svm_all_accs = []
svm_all_scores = []
svm_f1s = []
base_labels = []
lr_sensitivity= []
svm_sensitivity=[]
lr_specificity=[]
svm_specificity=[]

index=indices[0:int(max_features)]
features=X[:,index]
feature_num=features.shape[1]
for runs in range(1):
    lr_accuracies = []
    lr_scores = []
    svm_accuracies = []
    svm_scores = []
    strat_labels = []

    logistic = linear_model.LogisticRegression(C=1e5)
    rbf = svm.SVC(C=10,kernel='linear',gamma=0.01)
    
    skf=RepeatedStratifiedKFold(n_splits=10, n_repeats=10)
    for train_index, test_index in skf.split(features, y):
                train_data, test_data = features[train_index], features[test_index]
                train_labels, test_labels = y[train_index], y[test_index]
                #print(train_data.shape)
                strat_labels = np.append(strat_labels, test_labels)

                log = logistic.fit(train_data, train_labels)
                log_prob = log.decision_function(test_data)
                log_acc = log.score(test_data, test_labels)

                #f1 calculation
                y_pred = logistic.predict(test_data)
                log_f1 = f1_score(test_labels, y_pred)
                log_confuse= metrics.confusion_matrix(test_labels, y_pred)
                TP = log_confuse[1, 1]
                TN = log_confuse[0, 0]
                FP = log_confuse[0, 1]
                FN = log_confuse[1, 0]
                log_sen=metrics.recall_score(test_labels, y_pred)
    #             print(TN)
    #             print(FP)
                log_spec=TN / (TN + FP)
    #             print(log_confuse)
    #             print(log_sen)
    #             print(log_spec)

                lr_accuracies = np.append(lr_accuracies, log_acc)
                lr_sensitivity=np.append(lr_sensitivity, log_sen)
                lr_specificity=np.append(lr_specificity, log_spec)

                lr_scores = np.append(lr_scores, log_prob)
                lr_f1s = np.append(lr_f1s, log_f1)

                #print('Logistic Regression Accuracy: %f' % log_acc)

                rbf = rbf.fit(train_data, train_labels)
                svm_acc = rbf.score(test_data, test_labels)
                svm_prob = rbf.decision_function(test_data)

                #f1 calculation
                y_pred = rbf.predict(test_data)
                svm_f1 = f1_score(test_labels, y_pred)
                svm_confuse= metrics.confusion_matrix(test_labels, y_pred)
                TP = log_confuse[1, 1]
                TN = log_confuse[0, 0]
                FP = log_confuse[0, 1]
                FN = log_confuse[1, 0]
                svm_sen=metrics.recall_score(test_labels, y_pred)
                svm_spec=TN / (TN + FP)



                svm_accuracies = np.append(svm_accuracies, log_acc)
                svm_sensitivity=np.append(svm_sensitivity, log_sen)
                svm_specificity=np.append(svm_specificity, log_spec)

                #print('SVM Accuracy: %f' % svm_acc)
                svm_accuracies = np.append(svm_accuracies, svm_acc)
                svm_scores = np.append(svm_scores, svm_prob)
                svm_f1s = np.append(svm_f1s, svm_f1)

    base_labels = np.append(base_labels, strat_labels)
    lr_all_scores = np.append(lr_all_scores, lr_scores)
    lr_fold_avg = np.mean(lr_accuracies)
    lr_all_accs = np.append(lr_all_accs, lr_fold_avg)
    svm_all_scores = np.append(svm_all_scores, svm_scores)
    svm_fold_avg = np.mean(svm_accuracies)
    svm_all_accs = np.append(svm_all_accs, svm_fold_avg)

# feature_num_all=np.append(feature_num_all,feature_num)
# # print(np.mean(lr_all_accs))
print(np.mean(svm_all_accs))      
# lr_all_feature=np.append(lr_all_feature,np.mean(lr_all_accs))
# print(lr_all_feature)
# svm_all_feature=np.append(svm_all_feature,np.mean(svm_all_accs))
# print(np.mean(svm_accuracies))
# lr_fls_feature=np.append(lr_fls_feature,np.mean(lr_f1s))
# svm_fls_feature=np.append(svm_fls_feature,np.mean(svm_f1s))
#base_labels_all = np.append(base_labels_all,strat)
#base_labels = np.append(base_labels, np.mean(strat_labels))

print("LR accuracy Avg: {}".format(np.mean(lr_accuracies)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR accuracy Standard Deviation: {}".format(np.std(lr_accuracies)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR sensitivity Avg: {}".format(np.mean(lr_sensitivity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR sensitivity Standard Deviation: {}".format(np.std(lr_sensitivity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR specificity Avg: {}".format(np.mean(lr_specificity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR specificity Standard Deviation: {}".format(np.std(lr_specificity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR f1s Avg : {}".format(np.mean(lr_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("LR f1s Standard Deviation: {}".format(np.std(lr_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))

print("SVM Avg: {}".format(np.mean(svm_accuracies)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM Standard Deviation: {}".format(np.std(svm_accuracies)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM sensitivity Avg: {}".format(np.mean(svm_sensitivity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM sensitivity Standard Deviation: {}".format(np.std(svm_sensitivity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM specificity Avg: {}".format(np.mean(svm_specificity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM specificity Standard Deviation: {}".format(np.std(svm_specificity)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM f1s Avg : {}".format(np.mean(svm_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
print("SVM f1s Standard Deviation: {}".format(np.std(svm_f1s)),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
#lrindex=np.argmax(lr_all_feature)
print("LR Maximum feature number is: {}".format(feature_num_all[lrindex]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))
#svmindex=np.argmax(svm_all_feature)
print("SVM Maximum feature is number : {}".format(feature_num_all[svmindex]),file=open("/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/MCIvsSMIaccuracy.txt", "a"))


###############################################################################################################

from sklearn.metrics import roc_curve, auc,f1_score
import matplotlib.pyplot as plt

fpr = dict()
tpr = dict()
roc_auc = dict()

fpr['lr'], tpr['lr'], _ = roc_curve(base_labels, lr_all_scores)
roc_auc['lr'] = auc(fpr['lr'], tpr['lr'])
#f1['lr']=f1_score(fpr['lr'], tpr['lr'])
fpr['svm'], tpr['svm'], _ = roc_curve(base_labels, svm_all_scores)
roc_auc['svm'] = auc(fpr['svm'], tpr['svm'])
#f1['svm']=f1_score(fpr['svm'], tpr['svm'])
#fpr['gcn'], tpr['gcn'], _ = roc_curve(all_labels, all_scores)
#roc_auc['gcn'] = auc(fpr['gcn'], tpr['gcn'])

plt.figure()
plt.plot(fpr['lr'], tpr['lr'], lw=5, label='Logistic Regression (area = %0.2f)' % roc_auc['lr'] )
#plt.plot(fpr['lr'], tpr['lr'], lw=3, label='Logistic Regression (f1 = %0.2f)' % f1['lr'] )
plt.plot(fpr['svm'], tpr['svm'], lw=5, label='SVM (area = %0.2f)' % roc_auc['svm'] )
#plt.plot(fpr['svm'], tpr['svm'f], lw=3, label='SVM (f1 = %0.2f)' % f1['svm'] )
#plt.plot(fpr['gcn'], tpr['gcn'], lw=3, label='GCN (area = %0.2f)' % roc_auc['gcn'])
plt.plot([0, 1], [0, 1], color='navy', lw=5, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('MCIvsSMI')
plt.legend(loc="lower right") 
#plt.savefig('10x_Combined_ROC.eps')
#plt.savefig('ROC_MCIVsNormal_connectome.eps')
plt.show()

NameError: name 'np' is not defined

In [ ]:
plt.savefig('/Users/yunwang/Dropbox (NYSPI)/00 AD ML/0papers/0Neuroimage-Clinical/Figures/ROC_MCIvsSMI_connectome.pdf')